## Finally: A whole Knowledge Acquisition exercise

Think of the REMARO project example. Imagine having to repair a pipeline to an offshore terminal with your robots. You probably want to have a weather ontology that you can query for the best/ worst weather conditions to plan your work. This is exactly what we will do in this exercise!

What you will do here:
1. Create a basic weather data ontology
2. Choose a website with REST API to query current weather data for an offshore terminal
3. Write this data into the created ontology
4. Query the ontology for specific weather conditions to choose best times for your work

Note: If you start the Tutorials here, you need to install rdflib and owlready (`!pip install rdflib`, `!pip install owlready2`).

#### 1. Create a basic weather ontology

In [6]:
from owlready2 import *

# Create your own weather ontology
onto = get_ontology("http://purl.org/weather-data")

# Create the main classes, we will create a weather class
# let us define a wind_speed property for a weather as interesting property
with onto:
    class Weather(Thing):
        label = "weather at date time" 
    class wind_speed(DataProperty, Weather >> float, FunctionalProperty):
        label = "wind speed"   
        
list(Thing.subclasses())

[weather-data.Weather]

#### 2. Choose a website with REST API to query current weather data for an offshore terminal

There are many websites with weather data and REST API, for example:
-  https://developer.yr.no/doc/ (we will use this site in this exercise)
-  https://www.oceandatalab.com/home
-  https://earthexplorer.usgs.gov

For this example, let us use the "Mittelplatte" oil field, which has the coordinates 54.0259°N 8.731°E 
(for further info see https://de.wikipedia.org/wiki/Mittelplate_(%C3%96lfeld))

The following code accesses the REST API and writes the results into a json file:

In [7]:
import requests
import json

api_url = 'https://api.met.no/weatherapi/locationforecast/2.0/compact'
parameters = {'lat': 54,'lon': 8.73} #the previously defined coordinates
headers =  {
    'User-Agent': 'WeatherKG'
}
response = requests.get(api_url, headers = headers, params=parameters, timeout = 3)
# Assert that there is a response from the source
assert response.status_code == 200

In [5]:
with open('mittelplatte.json', 'w') as outfile:
    json.dump(response.json(),  outfile, indent=4)

You should now have created a new file named "mittelplatte" on the left. Check out the created .json file!
There is a time series of data with a wind speed property, amongst others. For now, we will only focus on this property.

#### 3. Write this data into the created ontology

For this, we open the created json file and retrieve the interesting parameters (in this case: wind speed) to write it into the corresponding fields in the ontology

In [12]:
import json

forecast = None
with open('mittelplatte.json') as file:
    forecast = json.load(file)  

# retrieve timeseries data
ts = forecast['properties']['timeseries']

# go through the file and write all weather instances into the ontology
# add the wind speed data property
for x in range(len(ts)):
    dtime = ts[x]['time']
    instance = Weather(dtime) 
    instant_details = ts[x]['data']['instant']['details']    
    instance.wind_speed = instant_details['wind_speed']        
        
# Save file
onto.save(file = "weather-data.owl", format = "rdfxml")   

You should now have created your own ontology named "weather-data.owl" on the left.

#### 4. Query the ontology for specific weather conditions to choose best times for your work

We can query the ontology for contained instances and their wind speed:

In [14]:
for i in Weather.instances(): print(i, i.wind_speed)

weather-data.2022-09-13T16:00:00Z 9.1
weather-data.2022-09-13T17:00:00Z 9.8
weather-data.2022-09-13T18:00:00Z 9.7
weather-data.2022-09-13T19:00:00Z 9.5
weather-data.2022-09-13T20:00:00Z 8.8
weather-data.2022-09-13T21:00:00Z 8.7
weather-data.2022-09-13T22:00:00Z 9.5
weather-data.2022-09-13T23:00:00Z 8.6
weather-data.2022-09-14T00:00:00Z 8.1
weather-data.2022-09-14T01:00:00Z 7.5
weather-data.2022-09-14T02:00:00Z 7.3
weather-data.2022-09-14T03:00:00Z 7.2
weather-data.2022-09-14T04:00:00Z 6.9
weather-data.2022-09-14T05:00:00Z 7.3
weather-data.2022-09-14T06:00:00Z 7.0
weather-data.2022-09-14T07:00:00Z 6.5
weather-data.2022-09-14T08:00:00Z 6.0
weather-data.2022-09-14T09:00:00Z 6.0
weather-data.2022-09-14T10:00:00Z 5.9
weather-data.2022-09-14T11:00:00Z 6.7
weather-data.2022-09-14T12:00:00Z 6.4
weather-data.2022-09-14T13:00:00Z 6.6
weather-data.2022-09-14T14:00:00Z 6.5
weather-data.2022-09-14T15:00:00Z 7.7
weather-data.2022-09-14T16:00:00Z 7.3
weather-data.2022-09-14T17:00:00Z 7.8
weather-data

Of course we can also filter out only instances that are above a defined number. If we know that conditions are too hard to perform tasks underwater at a wind speed above 5 , we can easily query only for times that are below this:

In [18]:
for i in Weather.instances(): 
    if i.wind_speed < 5:
      print(i, i.wind_speed)

weather-data.2022-09-20T12:00:00Z 4.8
weather-data.2022-09-20T18:00:00Z 4.8
weather-data.2022-09-21T00:00:00Z 4.5
weather-data.2022-09-21T06:00:00Z 3.3
weather-data.2022-09-21T12:00:00Z 3.4
weather-data.2022-09-21T18:00:00Z 4.5


Done! 

### Exercise 4

Get creative! 
You can either:
- add another interesting feature to the previously created ontology (a property, or a forecast) and query it
or
- query a new website like the oceandatalab and get whole new data (maybe wave height, or even ship routes?)

-> Last but not least: Im looking forward to you presenting your results to me :-)